In [1]:
import numpy as np
import open3d as o3d
import cv2
import json
import os
from pathlib import Path
from typing import List, Tuple, Dict
import trimesh

class DenseReconstruction:
    """
    Dense 3D reconstruction pipeline for room scene reconstruction
    from multi-view images and camera poses.
    """
    
    def __init__(self, data_folder: str):
        self.data_folder = Path(data_folder)
        self.images = []
        self.camera_params = []
        self.depth_maps = []
        self.point_clouds = []
        
    def load_camera_data(self, camera_file: str = "camera_poses.json"):
        """Load camera poses and intrinsics from JSON file"""
        camera_path = self.data_folder / camera_file
        
        if not camera_path.exists():
            print(f"Camera file not found. Looking for alternative formats...")
            # Try other common formats
            for fname in ['cameras.json', 'transforms.json', 'camera_movements.json']:
                alt_path = self.data_folder / fname
                if alt_path.exists():
                    camera_path = alt_path
                    break
        
        with open(camera_path, 'r') as f:
            data = json.load(f)
        
        self.camera_params = data
        return data
    
    def load_images(self, image_folder: str = "images"):
        """Load all images from the images folder"""
        img_path = self.data_folder / image_folder
        
        if not img_path.exists():
            img_path = self.data_folder
        
        image_files = sorted(list(img_path.glob("*.png")) + 
                           list(img_path.glob("*.jpg")) + 
                           list(img_path.glob("*.jpeg")))
        
        for img_file in image_files:
            img = cv2.imread(str(img_file))
            if img is not None:
                self.images.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                
        print(f"Loaded {len(self.images)} images")
        return self.images
    
    def estimate_depth_sgbm(self, img1: np.ndarray, img2: np.ndarray) -> np.ndarray:
        """Estimate depth using Semi-Global Block Matching"""
        gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
        gray2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
        
        # SGBM parameters
        window_size = 5
        min_disp = 0
        num_disp = 16 * 10  # Must be divisible by 16
        
        stereo = cv2.StereoSGBM_create(
            minDisparity=min_disp,
            numDisparities=num_disp,
            blockSize=window_size,
            P1=8 * 3 * window_size ** 2,
            P2=32 * 3 * window_size ** 2,
            disp12MaxDiff=1,
            uniquenessRatio=10,
            speckleWindowSize=100,
            speckleRange=32,
            preFilterCap=63,
            mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
        )
        
        disparity = stereo.compute(gray1, gray2).astype(np.float32) / 16.0
        
        # Convert disparity to depth
        focal_length = 500  # Approximate, adjust based on your camera
        baseline = 0.1  # Approximate baseline between views
        depth = (focal_length * baseline) / (disparity + 1e-10)
        depth[disparity <= 0] = 0
        
        return depth
    
    def depth_to_point_cloud(self, depth: np.ndarray, color: np.ndarray, 
                            intrinsics: Dict, extrinsics: np.ndarray = None) -> o3d.geometry.PointCloud:
        """Convert depth map to colored point cloud"""
        h, w = depth.shape
        
        # Camera intrinsics
        fx = intrinsics.get('fx', 500)
        fy = intrinsics.get('fy', 500)
        cx = intrinsics.get('cx', w / 2)
        cy = intrinsics.get('cy', h / 2)
        
        # Create meshgrid of pixel coordinates
        u, v = np.meshgrid(np.arange(w), np.arange(h))
        
        # Unproject to 3D
        z = depth
        x = (u - cx) * z / fx
        y = (v - cy) * z / fy
        
        # Filter out invalid depths
        valid = (z > 0) & (z < 10)  # Adjust max depth as needed
        
        points = np.stack([x[valid], y[valid], z[valid]], axis=-1)
        colors = color[valid] / 255.0
        
        # Create point cloud
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)
        pcd.colors = o3d.utility.Vector3dVector(colors)
        
        # Apply extrinsic transformation if provided
        if extrinsics is not None:
            pcd.transform(extrinsics)
        
        return pcd
    
    def create_dense_reconstruction(self, voxel_size: float = 0.01):
        """Main pipeline for dense reconstruction"""
        print("Starting dense reconstruction pipeline...")
        
        # Step 1: Load data
        self.load_images()
        
        # Default intrinsics if not provided
        default_intrinsics = {
            'fx': 500, 'fy': 500,
            'cx': self.images[0].shape[1] / 2 if self.images else 320,
            'cy': self.images[0].shape[0] / 2 if self.images else 240
        }
        
        # Step 2: Generate depth maps and point clouds
        print("Generating depth maps from image pairs...")
        for i in range(len(self.images) - 1):
            print(f"Processing image pair {i+1}/{len(self.images)-1}")
            
            depth = self.estimate_depth_sgbm(self.images[i], self.images[i+1])
            self.depth_maps.append(depth)
            
            # Convert to point cloud
            pcd = self.depth_to_point_cloud(depth, self.images[i], default_intrinsics)
            self.point_clouds.append(pcd)
        
        # Step 3: Merge all point clouds
        print("Merging point clouds...")
        merged_pcd = o3d.geometry.PointCloud()
        for pcd in self.point_clouds:
            merged_pcd += pcd
        
        # Step 4: Downsample
        print(f"Downsampling with voxel size {voxel_size}...")
        merged_pcd = merged_pcd.voxel_down_sample(voxel_size=voxel_size)
        
        # Step 5: Remove outliers
        print("Removing outliers...")
        merged_pcd, _ = merged_pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
        
        # Step 6: Estimate normals
        print("Estimating normals...")
        merged_pcd.estimate_normals(
            search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)
        )
        merged_pcd.orient_normals_consistent_tangent_plane(100)
        
        return merged_pcd
    
    def create_mesh_poisson(self, pcd: o3d.geometry.PointCloud, depth: int = 9):
        """Create mesh using Poisson surface reconstruction"""
        print("Creating mesh with Poisson reconstruction...")
        
        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
            pcd, depth=depth
        )
        
        # Remove low density vertices
        vertices_to_remove = densities < np.quantile(densities, 0.01)
        mesh.remove_vertices_by_mask(vertices_to_remove)
        
        return mesh
    
    def create_mesh_ball_pivoting(self, pcd: o3d.geometry.PointCloud):
        """Create mesh using Ball Pivoting Algorithm"""
        print("Creating mesh with Ball Pivoting...")
        
        radii = [0.005, 0.01, 0.02, 0.04]
        mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
            pcd, o3d.utility.DoubleVector(radii)
        )
        
        return mesh
    
    def save_results(self, pcd: o3d.geometry.PointCloud, mesh: o3d.geometry.TriangleMesh = None,
                    output_folder: str = "output"):
        """Save point cloud and mesh"""
        output_path = Path(output_folder)
        output_path.mkdir(exist_ok=True)
        
        # Save point cloud
        pcd_file = output_path / "dense_point_cloud.ply"
        o3d.io.write_point_cloud(str(pcd_file), pcd)
        print(f"Point cloud saved to {pcd_file}")
        
        # Save mesh if available
        if mesh is not None:
            mesh_file = output_path / "reconstructed_mesh.ply"
            o3d.io.write_triangle_mesh(str(mesh_file), mesh)
            print(f"Mesh saved to {mesh_file}")
    
    def visualize(self, geometries: List):
        """Visualize point clouds and meshes"""
        o3d.visualization.draw_geometries(
            geometries,
            window_name="3D Reconstruction",
            width=1024,
            height=768,
            left=50,
            top=50,
            point_show_normal=False
        )


def main():
    """Main execution function"""
    # Initialize reconstruction
    reconstructor = DenseReconstruction(data_folder="./data")
    
    # Try to load camera data
    try:
        reconstructor.load_camera_data()
    except FileNotFoundError:
        print("No camera data found, using default parameters")
    
    # Create dense reconstruction
    point_cloud = reconstructor.create_dense_reconstruction(voxel_size=0.01)
    
    print(f"Generated point cloud with {len(point_cloud.points)} points")
    
    # Create mesh (choose one method)
    try:
        mesh = reconstructor.create_mesh_poisson(point_cloud, depth=9)
        print(f"Generated mesh with {len(mesh.vertices)} vertices and {len(mesh.triangles)} triangles")
    except Exception as e:
        print(f"Poisson reconstruction failed: {e}")
        print("Trying Ball Pivoting instead...")
        mesh = reconstructor.create_mesh_ball_pivoting(point_cloud)
    
    # Save results
    reconstructor.save_results(point_cloud, mesh)
    
    # Visualize
    print("Visualizing results...")
    reconstructor.visualize([point_cloud, mesh])


if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'open3d'